### <center>Финальный проект курса "Введение в машинное обучение"</center>
#### <center>Предсказание победителя в онлайн-игре</center>                       

**Предметная область: Игра Dota 2**

Набор данных с матчами записан в файлах features.csv и features_test.csv.

### Подход 1: градиентный бустинг

In [1]:
import time
import datetime

In [3]:
import pandas as pd
data_train = pd.read_csv('C:./features.csv', index_col = 'match_id')
data_train.head(3)

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52.0,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5.0,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13.0,2130,0,0,1830,0,63


Удаляем признаки, связанные с итогами матча.

In [4]:
result_col = ['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 
              'barracks_status_radiant', 'barracks_status_dire']
X_train = data_train.drop(result_col, axis = 1)
X_train.shape

(97230, 102)

Проверим выборку на наличие пропусков с помощью функции count(), которая для каждого столбца показывает число заполненных значений. 

Запишем названия признаков, имеющих пропуски, и соответствующее им количество заполненных значений.

In [5]:
count = X_train.count()
for i in range(102):
    if(count[i] < 97230):
        print(count.index[i], count[i])

first_blood_time 77677
first_blood_team 77677
first_blood_player1 77677
first_blood_player2 53243
radiant_bottle_time 81539
radiant_courier_time 96538
radiant_flying_courier_time 69751
radiant_first_ward_time 95394
dire_bottle_time 81087
dire_courier_time 96554
dire_flying_courier_time 71132
dire_first_ward_time 95404


Заменяем пропуски на нули с помощью fillna().

In [6]:
X_train = X_train.fillna(0)

Столбец 'radiant_win' содержит целевую переменную.

In [7]:
y_train = data_train['radiant_win']

Фиксируем генератор разбиений для кросс-валидации по 5 блокам.

In [8]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)

Оценим качество градиентного бустинга (GradientBoostingClassifier) с помощью данной кросс-валидации.

Попробуем при этом разное количество деревьев - 10, 20, 30 и 40

In [9]:
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

lst_n_est = [10, 20, 30, 40]

for n_est in lst_n_est:
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_est, random_state=241)
    clf.fit(X_train, y_train)
    score = np.mean(cross_val_score(clf, X_train, y_train, scoring='roc_auc', cv=kf))
    print ("n_estimators = %s, Time elapsed = %s" % (n_est, datetime.datetime.now() - start_time))
    
    #val = round(scores.mean()*100,2)#берем среднее значение оценки
    print("n_estimators = %s, Score = %s" % (n_est,score))

n_estimators = 10, Time elapsed = 0:02:46.372793
n_estimators = 10, Score = 0.665780867531
n_estimators = 20, Time elapsed = 0:04:42.102043
n_estimators = 20, Score = 0.682417482077
n_estimators = 30, Time elapsed = 0:06:58.558655
n_estimators = 30, Score = 0.689601170713
n_estimators = 40, Time elapsed = 0:09:34.923976
n_estimators = 40, Score = 0.693818447332


**Мини-отчет для подхода 1: градиентный бустинг "в лоб"**

* Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

--> first_blood_time (Если событие "первая кровь" не успело произойти за первые 5 минут, то признаки принимают пропущенное значение)

--> first_blood_team 

--> first_blood_player1 

--> first_blood_player2 

--> radiant_bottle_time (Это  время первого приобретения командой предмета "bottle". За первые 5 минут команда могла и не приобрести предмет "bottle" вовсе)

--> radiant_courier_time

--> radiant_flying_courier_time 

--> radiant_first_ward_time 

--> dire_bottle_time 

--> dire_courier_time 

--> dire_flying_courier_time 

--> dire_first_ward_time 

* Как называется столбец, содержащий целевую переменную?

radiant_win: 1, если победила команда Radiant, 0 — иначе

* Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями?

Приблизительно 7 минут (время очень зависит от характеристик и работоспособности компьютера). 

* Имеет ли смысл использовать больше 30 деревьев в градиентном бустинге? Что бы вы предложили делать, чтобы ускорить его обучение при увеличении количества деревьев?

Время обучения увеличивается сильно с увеличением числа деревьев, а качество улучшается слабо.
Чтобы ускорить обучение можно уменьшить глубину деревьев и количество признаков, а также использовать признаки, имеющие максимальный вес при бустинге.

### Подход 2: логистическая регрессия

Оцениваем качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подбераем при этом лучший параметр регуляризации (C).

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid_C = {'C': np.linspace(0.001, 0.01, 25)}

def get_logreg_score(X, y):   

    logreg_grid = GridSearchCV(LogisticRegression(random_state=241), param_grid_C, cv=kf, 
                         scoring='roc_auc')
    logreg_grid.fit(X, y)
    print("Best parameter ", logreg_grid.best_params_)
    print("Best score ", logreg_grid.best_score_)   
    
    start_time = datetime.datetime.now()
    logreg = LogisticRegression(random_state=241, **logreg_grid.best_params_)
    logreg.fit(X, y)
    score_logreg = np.mean(cross_val_score(logreg, X, y, scoring='roc_auc', cv=kf))
    print ("Time elapsed = " ,  datetime.datetime.now() - start_time)

Качество для выборки, в которой признаки НЕ отмасшабированы:

In [11]:
get_logreg_score(X_train, y_train)

Best parameter  {'C': 0.001}
Best score  0.513449630822
Time elapsed =  0:00:08.651088


Качество для выборки, в которой признаки отмасшабированы:

In [12]:
from sklearn.preprocessing import StandardScaler
X_train_scaled = pd.DataFrame(data = StandardScaler().fit_transform(X_train))

get_logreg_score(X_train_scaled, y_train)

Best parameter  {'C': 0.0043750000000000004}
Best score  0.716164194651
Time elapsed =  0:00:37.340191


Заметим, что нормализация признаков значительно увеличила качество.

Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберем их из выборки, и проведем кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. 

In [13]:
categ_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                  'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
X_train_nocateg = X_train.drop(categ_features, axis = 1)
X_train_nocateg = X_train_nocateg.fillna(0)

X_train_nocateg_scaled = pd.DataFrame(data = StandardScaler().fit_transform(X_train_nocateg))
get_logreg_score(X_train_nocateg_scaled, y_train)

Best parameter  {'C': 0.0040000000000000001}
Best score  0.716212244225
Time elapsed =  0:00:35.170549


На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. 

Выясним из данных, сколько различных идентификаторов героев существует в данной игре.

In [14]:
categ_features.remove('lobby_type')
heroes = data_train[categ_features].values
n = np.unique(heroes)
n

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 109, 110, 112], dtype=int64)

In [15]:
n.size

108

Видим, что число РАЗЛИЧНЫХ идентификаторов героев = 108. Однако в выборке существуют также и такие идентификаторы, как 109, 110, 112. Это говорит о том, что есть пробелы, то есть идентификаторы принадлежат множеству натуральных чисел от 1 до 112, но среди идентификаторов не находятся числа 24, 107, 108, 111. 

Тем не менее, возьмем размерность нового фрейма X_pick = data_train.shape[0]*112.

В X_pick будут нулевые столбцы.

Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N (108, но возьмем 112) различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire.

In [16]:
N = 112
X_pick = np.zeros((data_train.shape[0], N))

for i, match_id in enumerate(data_train.index):
    for p in range(5):
        X_pick[i, data_train.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data_train.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

C:\Users\olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [17]:
X_pick_df = pd.DataFrame(data = X_pick)

Конкатенируем два набора дынных X_train_nocateg_scaled и X_pick_df - набор без категориальных признаков (с отмасштабированными переменными) и набор со 112 признаками, в которых закодирована информация о героях (категориальных признаках) с помощью "мешка слов".

In [18]:
X_train_bag = X_train_nocateg_scaled.join(X_pick_df, rsuffix='_')

Качество алгоритма на новой полученной выборке:

In [19]:
get_logreg_score(X_train_bag, y_train)

Best parameter  {'C': 0.01}
Best score  0.751417280128
Time elapsed =  0:01:01.358725


Заметим, что это самое высокое качество из всех полученных нами в ходе всего исследования. 

In [21]:
data_test = pd.read_csv('C:./features_test.csv', index_col = 'match_id')

Построим предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). 

**Лучшая модель - последняя (логистическая регрессия с l2 регуляризацией, коэффициент регуляризации = 0.01). Обучающий набор данных - X_train_bag.**

Для начала нужно обработать тестовую выборку (так же как и обучающую). 

In [22]:
categ_features = ['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                  'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']
X_test_nocateg = data_test.drop(categ_features, axis = 1)
X_test_nocateg = X_test_nocateg.fillna(0)
X_test_nocateg_scaled = pd.DataFrame(data = StandardScaler().fit_transform(X_test_nocateg))

categ_features.remove('lobby_type')
heroes_t = data_test[categ_features].values
n_t = np.unique(heroes_t)
n_t

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 109, 110, 112], dtype=int64)

In [23]:
N = 112
X_pick_test = np.zeros((data_test.shape[0], N))

for i, match_id in enumerate(data_test.index):
    for p in range(5):
        X_pick_test[i, data_test.loc[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, data_test.loc[match_id, 'd%d_hero' % (p+1)]-1] = -1

C:\Users\olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\olga\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [24]:
X_pick_test_df = pd.DataFrame(data = X_pick_test)
X_test_bag = X_test_nocateg_scaled.join(X_pick_test_df, rsuffix='_')

Строим модель:

In [25]:
lr = LogisticRegression(random_state=241, C = 0.01)
lr.fit(X_train_bag, y_train)

y_predictions = lr.predict_proba(X_test_bag)

Предсказания вероятностей победы команды Radiant:

In [26]:
y_predictions

array([[ 0.1860374 ,  0.8139626 ],
       [ 0.2546468 ,  0.7453532 ],
       [ 0.80497946,  0.19502054],
       ..., 
       [ 0.77095634,  0.22904366],
       [ 0.40456859,  0.59543141],
       [ 0.56049236,  0.43950764]])

In [ ]:
print(np.min(y_predictions), np.max(y_predictions))

Предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой.

np.min(y_predictions) = 0.0040168212460378605

np.max(y_predictions) = 0.99598317875396214

**Мини-отчет для подхода 2: логистическая регрессия**

* Какое качество получилось у логистической регрессии над всеми исходными признаками? Как оно соотносится с качеством градиентного бустинга? Быстрее ли работает логистическая регрессия по сравнению с градиентным бустингом?

-- Лучшие результаты для градиентного бустинга:
n_estimators = 40, Time elapsed = 0:09:34.923976
n_estimators = 40, Score = 0.693818447332

-- Лучшие результаты для логистической регрессии (признаки отмасштабированы). Результат до кодирования категориальных признаков с помощью "мешка слов":
Best score  0.716164194651
Time elapsed =  0:00:37.340191

(Вам может показаться, что время большое. Однако результат будет зависеть от характеристик Вашего компьютера. Если Вы будете прогонять код, то увидите, что результат получится другой. У меня слабый компьютер, поэтому время очень большое.)

По моим результатам, логистическая регрессия работает медленнее, но качество алгоритма выше.


* Сколько различных идентификаторов героев существует в данной игре?

108

* Какое получилось качество при добавлении "мешка слов" по героям? Улучшилось ли оно по сравнению с предыдущим вариантом? Чем вы можете это объяснить?

Best parameter  {'C': 0.01}
Best score  0.751417280128

Качество выросло. Когда мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду, мы потеряли действительно значимые признаки. Ведь это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. При добавлении "мешка слов" по героям, мы как раз корректно учли наши категориальные признаки. А следовательно и повысили качество.

* Какое минимальное и максимальное значение прогноза на тестовой выборке получилось у лучшего из алгоритмов?

np.min(y_predictions) = 0.0040168212460378605

np.max(y_predictions) = 0.99598317875396214

